In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Step 1: Load and prepare data
print("="*70)
print("ADVANCED STOCK PRICE PREDICTION WITH LSTM")
print("="*70)

ticker = 'AAL'
df = pd.read_csv("/content/SCOA_A5.csv")
df = df[df['Name'] == ticker].copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f"Loaded {len(df)} records for {ticker}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")

# Step 2: ADVANCED Feature Engineering
print("\n" + "="*70)
print("ADVANCED FEATURE ENGINEERING")
print("="*70)

# Basic features
df['price_change'] = df['close'].pct_change()
df['high_low_range'] = (df['high'] - df['low']) / df['close']
df['open_close_change'] = (df['close'] - df['open']) / df['open']

# Multiple Moving Averages
for window in [3, 5, 7, 10, 15, 20, 30]:
    df[f'ma_{window}'] = df['close'].rolling(window=window).mean()
    df[f'close_to_ma_{window}'] = (df['close'] - df[f'ma_{window}']) / df[f'ma_{window}']

# Exponential Moving Averages
df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()
df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = df['ema_12'] - df['ema_26']
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['macd_diff'] = df['macd'] - df['macd_signal']

# RSI with multiple periods
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

for period in [7, 14, 21]:
    df[f'rsi_{period}'] = compute_rsi(df['close'], period)

# Bollinger Bands
for window in [10, 20]:
    rolling_mean = df['close'].rolling(window=window).mean()
    rolling_std = df['close'].rolling(window=window).std()
    df[f'bb_upper_{window}'] = rolling_mean + (rolling_std * 2)
    df[f'bb_lower_{window}'] = rolling_mean - (rolling_std * 2)
    df[f'bb_position_{window}'] = (df['close'] - df[f'bb_lower_{window}']) / (df[f'bb_upper_{window}'] - df[f'bb_lower_{window}'])

# Volume analysis
df['volume_change'] = df['volume'].pct_change()
for window in [5, 10, 20]:
    df[f'volume_ma_{window}'] = df['volume'].rolling(window=window).mean()
    df[f'volume_ratio_{window}'] = df['volume'] / df[f'volume_ma_{window}']

# Volatility (multiple windows)
for window in [5, 10, 20, 30]:
    df[f'volatility_{window}'] = df['close'].rolling(window=window).std()
    df[f'volatility_ratio_{window}'] = df['close'].rolling(window=window).std() / df['close'].rolling(window=window).mean()

# Momentum indicators
for period in [3, 5, 10, 15, 20]:
    df[f'momentum_{period}'] = df['close'] - df['close'].shift(period)
    df[f'roc_{period}'] = df['close'].pct_change(periods=period)

# Price patterns
df['daily_return'] = df['close'].pct_change()
df['intraday_change'] = (df['close'] - df['open']) / df['open']
df['gap'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)

# Lagged features (crucial for sequence learning)
for lag in range(1, 11):  # Last 10 days
    df[f'close_lag_{lag}'] = df['close'].shift(lag)
    df[f'volume_lag_{lag}'] = df['volume'].shift(lag)
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)

# Target: Next day's closing price
df['Target'] = df['close'].shift(-1)

# Drop NaN
df_clean = df.dropna().copy()
print(f"After feature engineering: {len(df_clean)} records")

# Identify all feature columns (exclude date, Name, Target)
exclude_cols = ['date', 'Name', 'Target', 'open', 'high', 'low', 'close', 'volume']
feature_cols = [col for col in df_clean.columns if col not in exclude_cols]

print(f"Total features: {len(feature_cols)}")

X = df_clean[feature_cols].values
y = df_clean['Target'].values

# Step 3: Use RobustScaler (better for outliers in financial data)
from sklearn.preprocessing import RobustScaler

scaler_X = RobustScaler()
scaler_y = RobustScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Step 4: Create sequences for LSTM (look at past N days to predict next day)
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

print(f"\nSequence shape: {X_seq.shape}")
print(f"Each sample looks at {time_steps} days of data with {X_seq.shape[2]} features")

# Split data
test_size = 0.2
split_idx = int(len(X_seq) * (1 - test_size))

X_train = X_seq[:split_idx]
X_test = X_seq[split_idx:]
y_train = y_seq[:split_idx]
y_test = y_seq[split_idx:]

# Original prices for evaluation
y_train_original = scaler_y.inverse_transform(y_train.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

print(f"\nTrain samples: {len(X_train)} | Test samples: {len(X_test)}")

# Step 5: Build HYBRID LSTM-Dense Model
print("\n" + "="*70)
print("BUILDING HYBRID LSTM MODEL")
print("="*70)

model = Sequential([
    # Bidirectional LSTM layers (learn from past and future context)
    Bidirectional(LSTM(128, return_sequences=True, 
                       kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
                  input_shape=(time_steps, X_train.shape[2])),
    Dropout(0.3),
    
    Bidirectional(LSTM(64, return_sequences=True,
                       kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001))),
    Dropout(0.3),
    
    Bidirectional(LSTM(32, return_sequences=False,
                       kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001))),
    Dropout(0.2),
    
    # Dense layers
    Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)),
    Dropout(0.1),
    
    # Output
    Dense(1, activation='linear')
])

# Custom learning rate schedule
optimizer = Adam(learning_rate=0.0005)

model.compile(
    loss='huber',  # Huber loss is robust to outliers
    optimizer=optimizer,
    metrics=['mae']
)

model.summary()

# Step 6: Advanced Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=10,
    min_lr=0.00001,
    verbose=1
)

# Step 7: Train
print("\n" + "="*70)
print("TRAINING HYBRID MODEL")
print("="*70)

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=16,  # Smaller batch for better gradient estimates
    validation_split=0.15,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Step 8: Predictions
print("\n" + "="*70)
print("MODEL EVALUATION")
print("="*70)

y_train_pred_scaled = model.predict(X_train, verbose=0).flatten()
y_test_pred_scaled = model.predict(X_test, verbose=0).flatten()

y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

# Metrics
train_rmse = np.sqrt(mean_squared_error(y_train_original, y_train_pred))
train_mae = mean_absolute_error(y_train_original, y_train_pred)
train_r2 = r2_score(y_train_original, y_train_pred)
train_mape = np.mean(np.abs((y_train_original - y_train_pred) / y_train_original)) * 100

test_rmse = np.sqrt(mean_squared_error(y_test_original, y_test_pred))
test_mae = mean_absolute_error(y_test_original, y_test_pred)
test_r2 = r2_score(y_test_original, y_test_pred)
test_mape = np.mean(np.abs((y_test_original - y_test_pred) / y_test_original)) * 100

print("TRAINING SET METRICS:")
print(f"  RMSE: ${train_rmse:.4f}")
print(f"  MAE: ${train_mae:.4f}")
print(f"  R² Score: {train_r2:.4f}")
print(f"  MAPE: {train_mape:.2f}%")

print("\nTEST SET METRICS:")
print(f"  RMSE: ${test_rmse:.4f}")
print(f"  MAE: ${test_mae:.4f}")
print(f"  R² Score: {test_r2:.4f}")
print(f"  MAPE: {test_mape:.2f}%")

# Direction accuracy
train_direction_correct = np.sum((y_train_pred[1:] > y_train_pred[:-1]) == 
                                  (y_train_original[1:] > y_train_original[:-1]))
train_direction_acc = train_direction_correct / (len(y_train_pred) - 1) * 100

test_direction_correct = np.sum((y_test_pred[1:] > y_test_pred[:-1]) == 
                                 (y_test_original[1:] > y_test_original[:-1]))
test_direction_acc = test_direction_correct / (len(y_test_pred) - 1) * 100

print(f"\nDIRECTION ACCURACY:")
print(f"  Training: {train_direction_acc:.2f}%")
print(f"  Test: {test_direction_acc:.2f}%")

# Prediction bias analysis
train_bias = np.mean(y_train_pred - y_train_original)
test_bias = np.mean(y_test_pred - y_test_original)
print(f"\nPREDICTION BIAS (negative = underpredicting):")
print(f"  Training: ${train_bias:.4f}")
print(f"  Test: ${test_bias:.4f}")

# Step 9: Visualizations
print("\n" + "="*70)
print("GENERATING VISUALIZATIONS")
print("="*70)

fig = plt.figure(figsize=(18, 12))

# Plot 1: Loss
ax1 = plt.subplot(3, 3, 1)
ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_title('Model Loss Over Epochs', fontsize=11, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: MAE
ax2 = plt.subplot(3, 3, 2)
ax2.plot(history.history['mae'], label='Training MAE', linewidth=2)
ax2.plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
ax2.set_title('MAE Over Epochs', fontsize=11, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Actual vs Predicted
ax3 = plt.subplot(3, 3, 3)
ax3.scatter(y_test_original, y_test_pred, alpha=0.5, s=30)
min_val, max_val = y_test_original.min(), y_test_original.max()
ax3.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
ax3.set_title('Actual vs Predicted (Test)', fontsize=11, fontweight='bold')
ax3.set_xlabel('Actual Price ($)')
ax3.set_ylabel('Predicted Price ($)')
ax3.grid(True, alpha=0.3)

# Plot 4: Full time series
ax4 = plt.subplot(3, 3, 4)
ax4.plot(y_train_original, label='Train Actual', linewidth=1.5, alpha=0.7)
ax4.plot(y_train_pred, label='Train Predicted', linewidth=1.5, alpha=0.7)
ax4.set_title('Training Set Predictions', fontsize=11, fontweight='bold')
ax4.set_xlabel('Time')
ax4.set_ylabel('Price ($)')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Test predictions
ax5 = plt.subplot(3, 3, 5)
ax5.plot(y_test_original, label='Actual', linewidth=2, marker='o', markersize=3)
ax5.plot(y_test_pred, label='Predicted', linewidth=2, marker='s', markersize=3)
ax5.set_title('Test Set Predictions', fontsize=11, fontweight='bold')
ax5.set_xlabel('Time')
ax5.set_ylabel('Price ($)')
ax5.legend()
ax5.grid(True, alpha=0.3)

# Plot 6: Zoomed test predictions
ax6 = plt.subplot(3, 3, 6)
plot_points = min(50, len(y_test_original))
ax6.plot(range(plot_points), y_test_original[-plot_points:], 
         label='Actual', linewidth=2.5, marker='o', markersize=4)
ax6.plot(range(plot_points), y_test_pred[-plot_points:], 
         label='Predicted', linewidth=2.5, marker='s', markersize=4)
ax6.set_title(f'Last {plot_points} Days (Test)', fontsize=11, fontweight='bold')
ax6.set_xlabel('Days')
ax6.set_ylabel('Price ($)')
ax6.legend()
ax6.grid(True, alpha=0.3)

# Plot 7: Error distribution
ax7 = plt.subplot(3, 3, 7)
errors = y_test_original - y_test_pred
ax7.hist(errors, bins=50, alpha=0.7, edgecolor='black', color='steelblue')
ax7.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax7.axvline(x=np.mean(errors), color='g', linestyle='--', linewidth=2, label=f'Mean: ${np.mean(errors):.3f}')
ax7.set_title('Prediction Error Distribution', fontsize=11, fontweight='bold')
ax7.set_xlabel('Error ($)')
ax7.set_ylabel('Frequency')
ax7.legend()
ax7.grid(True, alpha=0.3)

# Plot 8: Residuals
ax8 = plt.subplot(3, 3, 8)
ax8.scatter(y_test_pred, errors, alpha=0.5, s=30)
ax8.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax8.set_title('Residual Plot', fontsize=11, fontweight='bold')
ax8.set_xlabel('Predicted Price ($)')
ax8.set_ylabel('Residual ($)')
ax8.grid(True, alpha=0.3)

# Plot 9: Prediction intervals
ax9 = plt.subplot(3, 3, 9)
plot_points = min(30, len(y_test_original))
x_range = range(plot_points)
ax9.plot(x_range, y_test_original[-plot_points:], 'go-', label='Actual', linewidth=2, markersize=6)
ax9.plot(x_range, y_test_pred[-plot_points:], 'bo-', label='Predicted', linewidth=2, markersize=6)
ax9.fill_between(x_range, 
                  y_test_pred[-plot_points:] - test_mae,
                  y_test_pred[-plot_points:] + test_mae,
                  alpha=0.2, label=f'±MAE (${test_mae:.3f})')
ax9.set_title('Prediction Confidence', fontsize=11, fontweight='bold')
ax9.set_xlabel('Days')
ax9.set_ylabel('Price ($)')
ax9.legend()
ax9.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_advanced_performance.png', dpi=300, bbox_inches='tight')
print("Saved: 'lstm_advanced_performance.png'")
plt.show()

# Sample predictions
print("\n" + "="*70)
print("SAMPLE PREDICTIONS (Last 15 test days)")
print("="*70)
print(f"{'Day':<6} {'Actual':<12} {'Predicted':<12} {'Error':<12} {'Error %':<10}")
print("-" * 60)
for i in range(-15, 0):
    actual = y_test_original[i]
    pred = y_test_pred[i]
    error = actual - pred
    error_pct = (error / actual) * 100
    print(f"{i+15:<6} ${actual:<11.2f} ${pred:<11.2f} ${error:<11.2f} {error_pct:<9.2f}%")

print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)